In [ ]:
!make clean

# Modern C++ part II: more than templates

1. Copy elision / RVO, move semantics
2. Variadic template and perfect forwarding
3. Anonymous function, closure, lambda expression

# Copy elision / return value optimization

Copy elision is one of the two forms of optimization, alongside allocation elision and extension, that is allowed to change the side effects.

Sometimes copy elision is also called return value optimization (RVO) or named return value optimization (NRVO).

In the following example (`03_elision/01_copy.cpp`), even though no optimization flag is used, the copy is optimized out:

```cpp
class IsCopied
{
public:
    static IsCopied & instance()
    {
        static IsCopied inst;
        return inst;
    }

    IsCopied & void on() { m_status = true; return *this; }
    operator bool() const { return m_status; }

private:
    IsCopied() : m_status(false) {}
    bool m_status;
};

class Data
{
public:
    constexpr const static size_t NELEM = 1024*8;
    Data()
    {
        std::cout << "Data constructed @" << this << std::endl;
    }
    Data(Data const & other)
    {
        copy_from(other);
        std::cout << "Data copied to @" << this << " from @" << &other << std::endl;
    }
    ~Data()
    {
        std::cout << "Data destructed @" << this << std::endl;
    }
    void copy_from(Data const & other)
    {
        for (size_t it=0; it < NELEM; ++it)
        {
            m_buffer[it] = other.m_buffer[it];
        }
        IsCopied::instance().on();
    }
};

void manipulate_with_reference(Data & data, int value)
{
    std::cout << "Manipulate with reference: " << &data << std::endl;

    for (size_t it=0; it < data.size(); ++it)
    {
        data[it] = value + it;
    }
    // In a real consumer function we will do much more meaningful operations.

    // However, we cannot destruct an object passed in with a reference.
}

Data worker1()
{
    Data data;

    // Manipulate the Data object.
    manipulate_with_reference(data, 3);

    return data;
}

Data worker2()
{
    Data data = worker1();

    // Manipulate the Data object, again.
    manipulate_with_reference(data, 8);

    return data;
}

int main(int argc, char ** argv)
{
    std::cout
        << (bool(IsCopied::instance()) ? "Something" : "Nothing")
        << " is copied" << std::endl;
    Data data = worker2();
    std::cout
        << (bool(IsCopied::instance()) ? "Something" : "Nothing")
        << " is copied" << std::endl;
}
```

In [ ]:
!make -C 03_elision clean ; make -C 03_elision OPT= 01_copy
#!r2 -Aqc "e scr.color=0 ; afl" 03_elision/01_copy
!r2 -Aqc "e scr.color=0 ; s sym.worker1 ; pdf ; s sym.worker2 ; pdf" 03_elision/01_copy

In [ ]:
!make -C 03_elision clean ; make -C 03_elision OPT= 01_copy
!03_elision/01_copy

# Move semantics and copy elision

Move semantics greatly helps us to avoid copying expensive resources.  To take advantage of that, our `Data` class should be changed to use dynamic allocation (`03_elision/02_move.cpp`):

```cpp
class Data
{

public:

    constexpr const static size_t NELEM = 1024*8;

    Data()
    {
        m_buffer = new int[NELEM];
        std::cout << "Data constructed @" << this << std::endl;
    }

    Data(Data const & other)
    {
        m_buffer = new int[NELEM];
        copy_from(other);
        std::cout << "Data copied to @" << this << " from @" << &other << std::endl;
    }

    Data & operator=(Data const & other)
    {
        if (nullptr == m_buffer) { m_buffer = new int[NELEM]; }
        copy_from(other);
        std::cout << "Data copy assigned to @" << this << " from @" << &other << std::endl;
        return *this;
    }

    Data(Data && other)
    {
        m_buffer = other.m_buffer;
        other.m_buffer = nullptr;
        std::cout << "Data moved to @" << this << " from @" << &other << std::endl;
        Status::instance().set_moved();
    }

    Data & operator=(Data && other)
    {
        if (m_buffer) { delete[] m_buffer; }
        m_buffer = other.m_buffer;
        other.m_buffer = nullptr;
        std::cout << "Data move assigned to @" << this << " from @" << &other << std::endl;
        Status::instance().set_moved();
        return *this;
    }

    ~Data()
    {
        if (m_buffer) { delete[] m_buffer; }
        std::cout << "Data destructed @" << this << std::endl;
    }

```

## Forced move is a bad idea

Although the move semantics indeed avoids copy the expensive buffer in the `Data` class, it cannot avoid copy the `Data` object itself.  However, copy elision (RVO & NRVO) can avoid copy the `Data` object.

```cpp
Data worker1()
{
    Data data;

    // Manipulate the Data object.
    manipulate_with_reference(data, 3);

    return data;
}

Data worker2()
{
    Data data = worker1();

    // Manipulate the Data object, again.
    manipulate_with_reference(data, 8);

#ifdef FORCEMOVE
    // Implicit move semantics destroys copy elision.
    return std::move(data);
#else
    return data;
#endif
}

int main(int argc, char ** argv)
{
    std::cout
        << "Status:"
        << (bool(Status::instance().is_copied()) ? " copied" : " uncopied")
        << (bool(Status::instance().is_moved()) ? " moved" : " unmoved")
        << std::endl;
    Data data = worker2();
    std::cout
        << "Status:"
        << (bool(Status::instance().is_copied()) ? " copied" : " uncopied")
        << (bool(Status::instance().is_moved()) ? " moved" : " unmoved")
        << std::endl;
}
```

In [ ]:
!make -C 03_elision clean ; make -C 03_elision FLAGS=-DFORCEMOVE OPT= 02_move
!03_elision/02_move

In [ ]:
!make -C 03_elision clean ; make -C 03_elision OPT= 02_move
!03_elision/02_move

# Data concatenation

Because of copy elision, for readibility in C++ it is prefer to write:

```cpp
std::vector<int> worker_return();
```

than

```cpp
void worker_argument(std::vector<int> & output /* output argument */);
```

Because in consumer code:

```cpp
// It reads clearly that the worker produces new result.
std::vector<int> result = worker_return();

// It takes a second to understand that the worker is using result as a buffer
// for output.
std::vector<int> result;
worker_argument(result);

/*
 * The result is pre-populated before sending to the worker.  From the
 * following lines we can't know how the worker will use result.
 *
 * By reading the worker signature we know that result may be used for output.
 * We can only be sure that result is used for output after reading the full
 * implemnetation of the worker.
 *
 * The worker may or may not expect the output argument to be pre-populated.
 * Regardless, it has to use runtime check to ensure either case.
 */
std::vector<int> result(100);
std::fill(result.begin(), result.end(), 7);
worker_argument(result);
```

The ambiguity is a productivity killer.  (Runtime performance is another story.)

## Style 1 of 3: return `vector`

In [ ]:
!make -C 03_elision clean ; make -C 03_elision FLAGS="-DOTYPE=1" 03_accumulate
!03_elision/03_accumulate

## Style 2: use output `vector`

In [ ]:
!make -C 03_elision clean ; make -C 03_elision FLAGS="-DMOVENOEXCEPT -DOTYPE=1" 03_accumulate
!03_elision/03_accumulate

## Style 3: use a class for both return and output argument

In [ ]:
!make -C 03_elision clean ; make -C 03_elision FLAGS="-DMOVENOEXCEPT -DOTYPE=2" 03_accumulate
!03_elision/03_accumulate

In [ ]:
!make -C 03_elision clean ; make -C 03_elision FLAGS="-DMOVENOEXCEPT -DOTYPE=3" 03_accumulate
!03_elision/03_accumulate

# Exercises

1. Measure the performance between using an output vector and returning a new vector.